In [1]:
# List contents of the working directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install py7zr;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 692.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 7.0 MB/s eta 0:00:00


In [3]:
# Import libraries

# Loading and processing
import glob
import cv2
import py7zr

# Data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
!nvidia-smi


/bin/bash: nvidia-smi: command not found


In [5]:
# Look for GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

GPU device not found
Found GPU at: 


Loading the data

In [6]:
# Class labels
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
# Open training archive file

if not len(glob.glob('/kaggle/working/train/*.png')):
    with py7zr.SevenZipFile('/kaggle/input/cifar-10/train.7z', mode='r') as z:
        z.extractall()

FileNotFoundError: ignored

In [8]:
# RGB pixel values lie on a [0,...,255] scale. We rescale the channels to have channel intensities lie 
# from 0 to 1 inclusive

X_train = X_train / 255.0

NameError: ignored

Visualization

In [ ]:
# To get a sense of the data we work with

plt.rcParams["figure.figsize"] = (10,10)
fig,axes = plt.subplots(10,10)
for i,ax in enumerate(axes.flatten()):
  ax.imshow(X_train[i])
  ax.set_title(f'{CLASS_NAMES[y_train[i]]}')
  ax.set_xticks([])
  ax.set_yticks([])
fig.tight_layout()


CNN Architecture

In [ ]:
# CNN model, with 6 convolutional layers, 3 pooling layers, and 3 dense layers. Softmax output layer.
model = models.Sequential()

model.add(layers.Conv2D(64, (5,5), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Augmenting the training data and training the model

In [ ]:
# We augment the training data within the mini-batches to try to reduce generalization error
datagen = ImageDataGenerator(
    horizontal_flip=True,
    channel_shift_range=0.05,
    rotation_range=10,
    validation_split=0.15
)

In [ ]:
# An example transformation

img = X_train[4]

fig, ax = plt.subplots(1,2)
ax[0].imshow(img)
ax[0].set_title("Original image")
ax[1].imshow(datagen.apply_transform(img,{'theta':-10,'channel_shift_intensity':0.05,'flip_horizontal':True}))
ax[1].set_title("Example transformation")

for i in range(2):
  ax[i].set_xticks([])
  ax[i].set_yticks([])
plt.show()

In [ ]:
# Train the model, including an early stopping if we can't reduce the loss further

early_stopping = keras.callbacks.EarlyStopping(monitor='loss',patience=10)

training_flow = datagen.flow(X_train, y_train, subset='training')
validation_flow = datagen.flow(X_train, y_train, subset='validation', shuffle=False)

history = model.fit(training_flow,
                    validation_data=validation_flow,
                    callbacks=[early_stopping],
                    epochs=75)


Model Evaluation

In [ ]:
# Plot the evolution of training/validation error and accuracy

train_metrics = history.history

fig, ax = plt.subplots(1,2)
ax[0].plot(train_metrics['loss'])
ax[0].plot(train_metrics['val_loss'])
ax[0].legend(['train','val'], loc='upper right')
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss")
ax[0].set_title("Loss over the training epochs")

ax[1].plot(train_metrics['accuracy'])
ax[1].plot(train_metrics['val_accuracy'])
ax[1].legend(['train','val'], loc='upper right')
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy")
ax[1].set_title("Accuracy over the training epochs")

plt.show()

In [ ]:
# Example predictions

sample_set = X_train[100:125]

sample_predictions = np.argmax(model.predict(sample_set),axis=1)

fig,axes = plt.subplots(5,5)
for i,ax in enumerate(axes.flatten()):
  ax.imshow(sample_set[i])
  ax.set_title(f'{CLASS_NAMES[sample_predictions[i]]}')
  ax.set_xticks([])
  ax.set_yticks([])
fig.tight_layout()

Predictions on the test set

In [ ]:
# Test set
if not len(glob.glob('/kaggle/working/test/*.png')):
    with py7zr.SevenZipFile('/kaggle/input/cifar-10/test.7z', mode='r') as z:
        z.extractall()

In [ ]:
# Feed in the files from the directory. We do not attempt to load all images to memory at once
test_datagen = ImageDataGenerator(rescale=1/255.0)

test_flow = test_datagen.flow_from_directory('/kaggle/working/test',
                                            classes=['.'],
                                            target_size=(32,32),
                                            shuffle=False)

In [ ]:
# Code -> classnames, and the helper function we made earlier
code_to_classname = {i:CLASS_NAMES[i] for i in range(len(CLASS_NAMES))}
def filename_to_id(filename):
    return int(filename.split('/')[-1].split('.')[0])

In [ ]:
# Form predictions
test_flow.reset()

test_output = model.predict(test_flow)
test_prediction_codes = pd.Series(np.argmax(test_output, axis=1))
test_prediction_labels = test_prediction_codes.map(code_to_classname)

test_filenames = pd.Series(test_flow.filenames)
test_ids = test_filenames.apply(filename_to_id)


In [ ]:
# Final cleaning up and output
out_df = pd.DataFrame({'id':test_ids,'label':test_prediction_labels})
out_df.sort_values(by=['id'], inplace=True)
out_df = out_df.reset_index(drop=True)
out_df = out_df.dropna()
out_df['id'] = out_df['id'].astype(int)

out_df.to_csv('/kaggle/working/submission.csv', index=False)